
# 10: Walk-Forward Validation Basics

このノートでは、時系列で重要な**walk-forward検証**を実装します。

構成:
1. 6か月学習 + 1か月テストをロール
2. ベースライン分類器（`scikit-learn` があれば Logistic Regression）
3. 未導入なら閾値ベース分類で代替
4. 精度(Accuracy)と簡易トレーディング指標（予測ポジション×翌日リターン）を評価



## なぜ walk-forward?

ランダム分割は未来情報が混ざりやすく、実運用の再現になりません。
walk-forward は時間順を守り、次のように進みます:

- Train: 6 months
- Test: 1 month
- 1 month 進めて繰り返し

> 注意: スケーラーなどの前処理は必ず train で fit し、test は transform のみ。


In [ ]:

import importlib.util
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from quantlab.ml_bridge import make_ml_table, iter_walk_forward_windows

np.random.seed(7)
dates = pd.date_range("2022-01-01", periods=560, freq="B")

# 教材用に緩いトレンド+ノイズを持つ疑似OHLCVを作成
drift = np.linspace(0, 12, len(dates))
close = pd.Series(120 + drift + np.cumsum(np.random.normal(0, 1.2, len(dates))), index=dates)
open_ = close.shift(1).fillna(close.iloc[0]) + np.random.normal(0, 0.5, len(dates))
spread = np.abs(np.random.normal(1.0, 0.25, len(dates)))
high = np.maximum(open_, close) + spread
low = np.minimum(open_, close) - spread
volume = (900_000 + np.random.normal(0, 100_000, len(dates))).clip(80_000)

ohlcv = pd.DataFrame({
    "Open": open_, "High": high, "Low": low, "Close": close, "Volume": volume
}, index=dates)

table = make_ml_table(ohlcv, label_method="next_day_direction")
feature_cols = [c for c in table.columns if c.startswith("ret_") or c in [
    "roll_mean_5", "roll_std_5", "roll_std_20", "ema_diff", "atr_14", "range_close", "volume_change_1d", "ema_diff_over_atr"
]]
label_col = [c for c in table.columns if c.startswith("label_")][0]

print("ML table:", table.shape)
print("Feature count:", len(feature_cols), "| label:", label_col)
table[[*feature_cols[:4], label_col]].head()


## Walk-forward split implementation (6m train / 1m test rolling)

In [ ]:

windows = list(iter_walk_forward_windows(table.index, train_months=6, test_months=1))
print("window count:", len(windows))
print("first window:", windows[0] if windows else "none")


## Baseline model: sklearn があれば Logistic Regression、なければ閾値分類

In [ ]:

    has_sklearn = importlib.util.find_spec("sklearn") is not None
    if not has_sklearn:
        print("scikit-learn is not installed. Optional install: pip install '.[ml]' または pip install scikit-learn")

    accuracies = []
    proxy_returns = []
    eval_points = []

    if has_sklearn:
        from sklearn.linear_model import LogisticRegression
        from sklearn.pipeline import Pipeline
        from sklearn.preprocessing import StandardScaler

    for w in windows:
        train_df = table.loc[w.train_start:w.train_end]
        test_df = table.loc[w.test_start:w.test_end]

        X_train = train_df[feature_cols]
        y_train = train_df[label_col].astype(int)
        X_test = test_df[feature_cols]
        y_test = test_df[label_col].astype(int)

        if has_sklearn:
            # leakage防止: scaler fitはtrainのみ
            model = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(max_iter=500, random_state=0)),
            ])
            model.fit(X_train, y_train)
            preds = model.predict(X_test)
        else:
            # 単純な閾値分類（ema_diff_over_atr > 0 なら上昇予測）
            preds = (X_test["ema_diff_over_atr"] > 0).astype(int).values

        acc = (preds == y_test.values).mean()

        # 予測ポジション: up予測なら+1, down予測なら-1
        pos = np.where(preds == 1, 1.0, -1.0)
        next_ret = ohlcv["Close"].pct_change().shift(-1).reindex(test_df.index).fillna(0.0)
        strat_ret = (pos * next_ret.values).mean()

        accuracies.append(float(acc))
        proxy_returns.append(float(strat_ret))
        eval_points.append(w.test_end)

    result = pd.DataFrame({
        "accuracy": accuracies,
        "proxy_mean_return": proxy_returns,
    }, index=pd.to_datetime(eval_points))

    print(result.head())
    print("
Average accuracy:", round(result["accuracy"].mean(), 4))
    print("Average proxy return:", round(result["proxy_mean_return"].mean(), 6))


## 可視化: 各テスト期間の Accuracy / Proxy Return

In [ ]:

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
result["accuracy"].plot(ax=axes[0], marker="o", title="Walk-Forward Accuracy")
axes[0].axhline(result["accuracy"].mean(), color="gray", linestyle="--", label="mean")
axes[0].legend()

result["proxy_mean_return"].plot(ax=axes[1], marker="o", color="tab:green", title="Walk-Forward Proxy Mean Return")
axes[1].axhline(0.0, color="black", linewidth=1)

plt.tight_layout()
plt.show()



## What to observe

- 同じ戦略でも、期間によって精度・期待値が大きく変化します（regime依存）。
- Accuracyが高くても収益代理が弱いケースがあるため、複数指標で評価します。
- `scikit-learn` 利用時は `Pipeline(StandardScaler -> LogisticRegression)` で漏洩を防ぎやすくなります。

関連ドキュメント: `docs/03_ml_bridge_feature_label_validation.md`
